In [1]:
import psycopg2
import pandas as pds
from sqlalchemy import create_engine
import datetime
from typing import List, Tuple
import random

## Extracción de los datos que se generaron con mockaroo

In [2]:
DATABASE = "proyecto"
PORT="5432"
HOST="localhost"
USERNAME="alexndf"
PASSWORD="postgrespw123"

alchemy_engine   = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')
db_connection = alchemy_engine.connect()

agente = pds.read_sql("SELECT * FROM agente", db_connection)
empleado = pds.read_sql("SELECT * FROM empleado", db_connection)
curso = pds.read_sql("SELECT * FROM curso", db_connection)
sala_capacitacion = pds.read_sql("SELECT * FROM sala_capacitacion", db_connection)
requerir = pds.read_sql("SELECT * FROM requerir", db_connection)
evaluar = pds.read_sql("SELECT * FROM evaluar", db_connection)
reservacion_operaciones = pds.read_sql("SELECT * FROM reservacion_operaciones", db_connection)

db_connection.close()

## Métodos auxiliares

In [3]:
START_DATE = datetime.date(2022, 9, 1)
CURRENT_DATE = datetime.date(2022, 12, 5)
END_DATE = datetime.date(2022, 12, 18)
random.seed(1)

In [4]:
def get_time_str(time:datetime.time):
    return time.strftime('%H:%M:%S')

def get_postgresql_repr(v):
    if isinstance(v, datetime.time):
        return f"'{v.strftime('%H:%M:%S')}'::time"
    elif isinstance(v, datetime.date):
        return f"'{v.isoformat()}'::date"
    elif isinstance(v, datetime.datetime):
        s= v.strftime("%Y-%m-%d, %H:%M:%S")
        return f"'{s}'::timestamp"
    elif isinstance(v, tuple) and list(map(type, v))==[datetime.datetime, datetime.datetime]:
        dt1, dt2 = v
        s1, s2 = dt1.strftime("%Y-%m-%d %H:%M:%S"), dt2.strftime("%Y-%m-%d %H:%M:%S")
        return f"tsrange('({s1}, {s2})')"
    elif isinstance(v, str):
        return f"'{v}'"
    else:
        return str(v)

def get_sql_insert_statements_from_dataframe(SOURCE, TARGET):
    sql_texts = []
    cols_str = str(", ".join(SOURCE.columns))
    for index, row in SOURCE.iterrows():
        row_values_str = "(" + ",".join(map(get_postgresql_repr, row.values)) + ")"
        sql_texts.append(
            f"INSERT INTO {TARGET} ({cols_str}) VALUES {row_values_str};"
        )
    return sql_texts


In [5]:
def get_work_days_in_range(d1:datetime.date, d2:datetime.date)->List[datetime.date]:
    size = (d2-d1).days + 1
    dias = []
    for i in range(size):
        d = d1+datetime.timedelta(days=i)
        if d.weekday()!=6: 
            dias.append(d)
    return dias

In [6]:
def random_time(start:datetime.time, end:datetime.time):
    max_duration_seconds = (((end.hour-start.hour)*60 + (end.minute-start.minute))*60 +  (end.second-start.second))
    duration=random.randrange(0, max_duration_seconds+1)
    time_today = datetime.datetime.combine(datetime.date.today(), start) + datetime.timedelta(seconds=duration)
    return time_today.time()

## Generación de los datos en dataframes de Pandas

### Relaciones de asistencia

`asistencia`

In [7]:
asistencia_cols = ["id_empleado","num_piso", "id_edificio"]
asistencia = empleado[asistencia_cols]
display(asistencia)

,id_empleado,num_piso,id_edificio
0,1,1,1
1,2,1,2
2,3,1,3
3,4,1,4
4,5,1,5
5,6,2,1
6,7,2,2
7,8,2,3
8,9,2,4
9,10,2,5


`registro_asistencia`

In [8]:
registro_asistencia_cols = "id_empleado,num_piso,id_edificio,fecha,hora,tipo".split(",")
registro_asistencia_rows = []
for fecha in get_work_days_in_range(START_DATE, CURRENT_DATE):
    for i, r in asistencia.iterrows():
        id_empleado = r["id_empleado"]
        num_piso = r["num_piso"]
        id_edificio = r["id_edificio"]

        hora_entrada = random_time(datetime.time(7), datetime.time(10))
        registro_asistencia_rows.append(
            [id_empleado, num_piso, id_edificio, fecha, hora_entrada, "entrada"]
        )

        hora_salida = random_time(datetime.time(18), datetime.time(19))
        registro_asistencia_rows.append(
            [id_empleado, num_piso, id_edificio, fecha, hora_salida, "salida"]
        )
registro_asistencia = pds.DataFrame(
    registro_asistencia_rows, columns=registro_asistencia_cols
)
display(len(registro_asistencia))
display(registro_asistencia)


8200

,id_empleado,num_piso,id_edificio,fecha,hora,tipo
0,1,1,1,2022-09-01,07:36:41,entrada
1,1,1,1,2022-09-01,18:38:51,salida
2,2,1,2,2022-09-01,07:17:13,entrada
3,2,1,2,2022-09-01,18:17:24,salida
4,3,1,3,2022-09-01,07:32:11,entrada
...,...,...,...,...,...,...
8195,48,5,3,2022-12-05,18:45:09,salida
8196,49,5,4,2022-12-05,07:54:38,entrada
8197,49,5,4,2022-12-05,18:26:34,salida
8198,50,5,5,2022-12-05,08:30:39,entrada


### Relaciones de salas de capacitación

`horario_curso`

In [9]:
horario_curso_cols = "id_curso,id_programa_curso,id_cliente,rango".split(",")
horario_curso_rows = []


fechas = get_work_days_in_range(START_DATE, CURRENT_DATE)
for i, r in curso.iterrows():
    horas_por_dia_semana = {
        i:(datetime.time(random.randrange(12, 14+1)), datetime.time(random.randrange(15, 18+1)))
        for i in range(6)
    }
    for fecha in fechas:
        id_curso = r["id_curso"]
        id_programa_curso = r["id_programa_curso"]
        id_cliente = r["id_cliente"]

        dia_semana = fecha.weekday()
        hora_comienzo, hora_fin = horas_por_dia_semana[dia_semana]
        rango = (datetime.datetime.combine(fecha, hora_comienzo), datetime.datetime.combine(fecha, hora_fin))

        horario_curso_rows.append(
            [id_curso, id_programa_curso, id_cliente, rango]
        )
horario_curso = pds.DataFrame(horario_curso_rows, columns=horario_curso_cols)
display(horario_curso)

,id_curso,id_programa_curso,id_cliente,rango
0,1,1,200,"(2022-09-01 14:00:00, 2022-09-01 15:00:00)"
1,1,1,200,"(2022-09-02 14:00:00, 2022-09-02 15:00:00)"
2,1,1,200,"(2022-09-03 14:00:00, 2022-09-03 18:00:00)"
3,1,1,200,"(2022-09-05 14:00:00, 2022-09-05 18:00:00)"
4,1,1,200,"(2022-09-06 13:00:00, 2022-09-06 17:00:00)"
...,...,...,...,...
3275,40,40,239,"(2022-11-30 14:00:00, 2022-11-30 18:00:00)"
3276,40,40,239,"(2022-12-01 12:00:00, 2022-12-01 17:00:00)"
3277,40,40,239,"(2022-12-02 14:00:00, 2022-12-02 15:00:00)"
3278,40,40,239,"(2022-12-03 14:00:00, 2022-12-03 16:00:00)"


`asignar` (salas de capacitación)

In [10]:
salas_por_piso_y_edificio = dict()
for _, row in sala_capacitacion.iterrows():
    num_sala = row["num_sala"]
    num_piso = row["num_piso"]
    id_edificio = row["id_edificio"]
    
    piso_edificio = (num_piso, id_edificio)
    if piso_edificio not in salas_por_piso_y_edificio:
        salas_por_piso_y_edificio[piso_edificio]=list()

    salas_por_piso_y_edificio[piso_edificio].append(num_sala)

agente_sala_curso_presencial = pds.merge(
    curso[curso["modalidad"] == "presencial"][["id_curso", "id_programa_curso", "id_cliente"]],
    agente[["id_empleado", "id_curso", "id_programa_curso", "id_cliente", "num_piso", "id_edificio"]],
    how="inner",
)
col_salas=list()
for _, row in agente_sala_curso_presencial.iterrows():
    num_piso = row["num_piso"]	
    id_edificio = row["id_edificio"]

    salas_disponibles = salas_por_piso_y_edificio[(num_piso, id_edificio)]
    i_sala = random.randrange(0, len(salas_disponibles))
    num_sala = salas_disponibles[i_sala]
    salas_disponibles[i_sala], salas_disponibles[-1]=salas_disponibles[-1],salas_disponibles[i_sala]
    salas_disponibles.pop()

    col_salas.append(num_sala)
agente_sala_curso_presencial["num_sala"]=col_salas

display(agente_sala_curso_presencial)

,id_curso,id_programa_curso,id_cliente,id_empleado,num_piso,id_edificio,num_sala
0,2,2,201,2,1,2,1
1,5,5,204,5,1,5,1
2,8,8,207,8,2,3,1
3,10,10,209,10,2,5,1
4,12,12,211,12,3,2,1
5,13,13,212,13,3,3,1
6,16,16,215,16,4,1,1
7,18,18,217,18,4,3,1
8,20,20,219,20,4,5,1
9,23,23,222,23,5,3,1


In [11]:
horario_curso_agente_sala_curso_presencial = pds.merge(
    horario_curso,
    agente_sala_curso_presencial,
    how="inner",
    on=["id_curso","id_programa_curso","id_cliente"]
)
horario_curso_agente_sala_curso_presencial

,id_curso,id_programa_curso,id_cliente,rango,id_empleado,num_piso,id_edificio,num_sala
0,2,2,201,"(2022-09-01 14:00:00, 2022-09-01 16:00:00)",2,1,2,1
1,2,2,201,"(2022-09-02 12:00:00, 2022-09-02 17:00:00)",2,1,2,1
2,2,2,201,"(2022-09-03 14:00:00, 2022-09-03 18:00:00)",2,1,2,1
3,2,2,201,"(2022-09-05 14:00:00, 2022-09-05 16:00:00)",2,1,2,1
4,2,2,201,"(2022-09-06 12:00:00, 2022-09-06 18:00:00)",2,1,2,1
...,...,...,...,...,...,...,...,...
815,23,23,222,"(2022-11-30 12:00:00, 2022-11-30 16:00:00)",23,5,3,1
816,23,23,222,"(2022-12-01 12:00:00, 2022-12-01 18:00:00)",23,5,3,1
817,23,23,222,"(2022-12-02 14:00:00, 2022-12-02 16:00:00)",23,5,3,1
818,23,23,222,"(2022-12-03 14:00:00, 2022-12-03 18:00:00)",23,5,3,1


In [12]:
asignar_cols = "num_sala,num_piso,id_edificio,id_curso,id_programa_curso,id_cliente,horario_reserva".split(
    ","
)
asignar_rows = []

for _, row in horario_curso_agente_sala_curso_presencial.iterrows():
    num_sala = row["num_sala"]
    num_piso = row["num_piso"]
    id_edificio = row["id_edificio"]
    id_curso = row["id_curso"]
    id_programa_curso = row["id_programa_curso"]
    id_cliente = row["id_cliente"]
    horario_reserva=row["rango"]

    asignar_rows.append(
        [
            num_sala,
            num_piso,
            id_edificio,
            id_curso,
            id_programa_curso,
            id_cliente,
            horario_reserva,
        ]
    )

asignar=pds.DataFrame(asignar_rows, columns=asignar_cols)
display(asignar)


,num_sala,num_piso,id_edificio,id_curso,id_programa_curso,id_cliente,horario_reserva
0,1,1,2,2,2,201,"(2022-09-01 14:00:00, 2022-09-01 16:00:00)"
1,1,1,2,2,2,201,"(2022-09-02 12:00:00, 2022-09-02 17:00:00)"
2,1,1,2,2,2,201,"(2022-09-03 14:00:00, 2022-09-03 18:00:00)"
3,1,1,2,2,2,201,"(2022-09-05 14:00:00, 2022-09-05 16:00:00)"
4,1,1,2,2,2,201,"(2022-09-06 12:00:00, 2022-09-06 18:00:00)"
...,...,...,...,...,...,...,...
815,1,5,3,23,23,222,"(2022-11-30 12:00:00, 2022-11-30 16:00:00)"
816,1,5,3,23,23,222,"(2022-12-01 12:00:00, 2022-12-01 18:00:00)"
817,1,5,3,23,23,222,"(2022-12-02 14:00:00, 2022-12-02 16:00:00)"
818,1,5,3,23,23,222,"(2022-12-03 14:00:00, 2022-12-03 18:00:00)"


`faltar`

In [13]:
faltar_cols = "id_empleado,id_curso,id_programa_curso,id_cliente,fecha".split(",")
faltar_rows = []

for _, row in agente.iterrows():
    for fecha in get_work_days_in_range(START_DATE, END_DATE):
        id_curso = row["id_curso"]

        if id_curso is not None and random.randrange(1, 101) == 1:
            id_empleado = row["id_empleado"]
            id_programa_curso = row["id_programa_curso"]
            id_cliente = row["id_cliente"]

            faltar_rows.append(
                [id_empleado, id_curso, id_programa_curso, id_cliente, fecha]
            )

faltar=pds.DataFrame(faltar_rows, columns=faltar_cols)
display(faltar)

,id_empleado,id_curso,id_programa_curso,id_cliente,fecha
0,2,2,2,201,2022-11-15
1,2,2,2,201,2022-11-26
2,3,3,3,202,2022-09-22
3,3,3,3,202,2022-09-29
4,4,4,4,203,2022-11-16
5,5,5,5,204,2022-12-16
6,7,7,7,206,2022-09-17
7,9,9,9,208,2022-12-14
8,10,10,10,209,2022-10-26
9,12,12,12,211,2022-10-03


### Relaciones de salas de capacitación

`fecha_reservacion_operaciones`

In [14]:
fecha_reservacion_operaciones_cols = "id_reservacion_operaciones,num_sala,num_piso,id_edificio,fecha".split(",")
fecha_reservacion_operaciones_rows = []

for _, row in reservacion_operaciones.iterrows():
    for fecha in get_work_days_in_range(START_DATE, END_DATE):
        id_reservacion_operaciones=row["id_reservacion_operaciones"]
        num_sala=row["num_sala"]
        num_piso=row["num_piso"]
        id_edificio=row["id_edificio"]

        fecha_reservacion_operaciones_rows.append(
            [id_reservacion_operaciones,num_sala,num_piso,id_edificio,fecha]
        )
fecha_reservacion_operaciones=pds.DataFrame(fecha_reservacion_operaciones_rows, columns=fecha_reservacion_operaciones_cols)
display(fecha_reservacion_operaciones)

,id_reservacion_operaciones,num_sala,num_piso,id_edificio,fecha
0,1,2,1,1,2022-09-01
1,1,2,1,1,2022-09-02
2,1,2,1,1,2022-09-03
3,1,2,1,1,2022-09-05
4,1,2,1,1,2022-09-06
...,...,...,...,...,...
3715,40,2,10,4,2022-12-13
3716,40,2,10,4,2022-12-14
3717,40,2,10,4,2022-12-15
3718,40,2,10,4,2022-12-16


`laborar_operaciones`

In [15]:
reservacion_agentes_que_aprobaron_requisitos = pds.merge(
    reservacion_operaciones,
    pds.merge(
        evaluar[evaluar["calificacion"]>=8],
        requerir,
        how="inner",
        on="id_programa_curso"
    ),
    how="inner",
    on="id_reservacion_operaciones,num_sala,num_piso,id_edificio".split(",")
)
reservacion_agentes_que_aprobaron_requisitos

,id_reservacion_operaciones,num_sala,num_piso,id_edificio,sistema_operativo,id_empleado,id_curso,id_programa_curso,id_cliente_x,calificacion,id_cliente_y
0,1,2,1,1,Windows,1,1,1,200,9,200
1,2,2,2,1,Windows,2,2,2,201,8,201
2,3,2,3,1,Linux,3,3,3,202,10,202
3,4,2,4,1,Linux,4,4,4,203,10,203
4,5,2,5,1,Linux,5,5,5,204,10,204
5,6,2,6,1,Windows,6,6,6,205,8,205
6,7,2,7,1,Linux,7,7,7,206,10,206
7,8,2,8,1,Linux,8,8,8,207,10,207
8,9,2,9,1,Windows,9,9,9,208,9,208
9,10,2,10,1,Windows,10,10,10,209,8,209


In [16]:
laborar_operaciones_cols = "id_reservacion_operaciones,num_sala,num_piso,id_edificio,id_empleado".split(",")
laborar_operaciones_rows = []

for _, row in reservacion_agentes_que_aprobaron_requisitos.iterrows():
    id_reservacion_operaciones=row["id_reservacion_operaciones"]
    num_sala=row["num_sala"]
    num_piso=row["num_piso"]
    id_edificio=row["id_edificio"]
    id_empleado=row["id_empleado"]

    laborar_operaciones_rows.append(
        [id_reservacion_operaciones,num_sala,num_piso,id_edificio,id_empleado]
    )

laborar_operaciones=pds.DataFrame(laborar_operaciones_rows, columns=laborar_operaciones_cols)
display(laborar_operaciones)

,id_reservacion_operaciones,num_sala,num_piso,id_edificio,id_empleado
0,1,2,1,1,1
1,2,2,2,1,2
2,3,2,3,1,3
3,4,2,4,1,4
4,5,2,5,1,5
5,6,2,6,1,6
6,7,2,7,1,7
7,8,2,8,1,8
8,9,2,9,1,9
9,10,2,10,1,10


## Conversión de los datos a cláusulas _insert_ de SQL

In [17]:
asistencia_inserts = get_sql_insert_statements_from_dataframe(asistencia, "asistencia")
display(asistencia_inserts[:5])

['INSERT INTO asistencia (id_empleado, num_piso, id_edificio) VALUES (1,1,1);',
 'INSERT INTO asistencia (id_empleado, num_piso, id_edificio) VALUES (2,1,2);',
 'INSERT INTO asistencia (id_empleado, num_piso, id_edificio) VALUES (3,1,3);',
 'INSERT INTO asistencia (id_empleado, num_piso, id_edificio) VALUES (4,1,4);',
 'INSERT INTO asistencia (id_empleado, num_piso, id_edificio) VALUES (5,1,5);']

In [18]:
registro_asistencia_inserts = get_sql_insert_statements_from_dataframe(registro_asistencia, "registro_asistencia")
display(registro_asistencia_inserts[:5])

["INSERT INTO registro_asistencia (id_empleado, num_piso, id_edificio, fecha, hora, tipo) VALUES (1,1,1,'2022-09-01'::date,'07:36:41'::time,'entrada');",
 "INSERT INTO registro_asistencia (id_empleado, num_piso, id_edificio, fecha, hora, tipo) VALUES (1,1,1,'2022-09-01'::date,'18:38:51'::time,'salida');",
 "INSERT INTO registro_asistencia (id_empleado, num_piso, id_edificio, fecha, hora, tipo) VALUES (2,1,2,'2022-09-01'::date,'07:17:13'::time,'entrada');",
 "INSERT INTO registro_asistencia (id_empleado, num_piso, id_edificio, fecha, hora, tipo) VALUES (2,1,2,'2022-09-01'::date,'18:17:24'::time,'salida');",
 "INSERT INTO registro_asistencia (id_empleado, num_piso, id_edificio, fecha, hora, tipo) VALUES (3,1,3,'2022-09-01'::date,'07:32:11'::time,'entrada');"]

In [19]:
horario_curso_inserts = get_sql_insert_statements_from_dataframe(horario_curso, "horario_curso")
display(horario_curso_inserts[:5])

["INSERT INTO horario_curso (id_curso, id_programa_curso, id_cliente, rango) VALUES (1,1,200,tsrange('(2022-09-01 14:00:00, 2022-09-01 15:00:00)'));",
 "INSERT INTO horario_curso (id_curso, id_programa_curso, id_cliente, rango) VALUES (1,1,200,tsrange('(2022-09-02 14:00:00, 2022-09-02 15:00:00)'));",
 "INSERT INTO horario_curso (id_curso, id_programa_curso, id_cliente, rango) VALUES (1,1,200,tsrange('(2022-09-03 14:00:00, 2022-09-03 18:00:00)'));",
 "INSERT INTO horario_curso (id_curso, id_programa_curso, id_cliente, rango) VALUES (1,1,200,tsrange('(2022-09-05 14:00:00, 2022-09-05 18:00:00)'));",
 "INSERT INTO horario_curso (id_curso, id_programa_curso, id_cliente, rango) VALUES (1,1,200,tsrange('(2022-09-06 13:00:00, 2022-09-06 17:00:00)'));"]

In [20]:
asignar_inserts = get_sql_insert_statements_from_dataframe(asignar, "asignar")
display(asignar_inserts[:5])

["INSERT INTO asignar (num_sala, num_piso, id_edificio, id_curso, id_programa_curso, id_cliente, horario_reserva) VALUES (1,1,2,2,2,201,tsrange('(2022-09-01 14:00:00, 2022-09-01 16:00:00)'));",
 "INSERT INTO asignar (num_sala, num_piso, id_edificio, id_curso, id_programa_curso, id_cliente, horario_reserva) VALUES (1,1,2,2,2,201,tsrange('(2022-09-02 12:00:00, 2022-09-02 17:00:00)'));",
 "INSERT INTO asignar (num_sala, num_piso, id_edificio, id_curso, id_programa_curso, id_cliente, horario_reserva) VALUES (1,1,2,2,2,201,tsrange('(2022-09-03 14:00:00, 2022-09-03 18:00:00)'));",
 "INSERT INTO asignar (num_sala, num_piso, id_edificio, id_curso, id_programa_curso, id_cliente, horario_reserva) VALUES (1,1,2,2,2,201,tsrange('(2022-09-05 14:00:00, 2022-09-05 16:00:00)'));",
 "INSERT INTO asignar (num_sala, num_piso, id_edificio, id_curso, id_programa_curso, id_cliente, horario_reserva) VALUES (1,1,2,2,2,201,tsrange('(2022-09-06 12:00:00, 2022-09-06 18:00:00)'));"]

In [21]:
faltar_inserts = get_sql_insert_statements_from_dataframe(faltar, "faltar")
display(faltar_inserts[:5])

["INSERT INTO faltar (id_empleado, id_curso, id_programa_curso, id_cliente, fecha) VALUES (2,2,2,201,'2022-11-15'::date);",
 "INSERT INTO faltar (id_empleado, id_curso, id_programa_curso, id_cliente, fecha) VALUES (2,2,2,201,'2022-11-26'::date);",
 "INSERT INTO faltar (id_empleado, id_curso, id_programa_curso, id_cliente, fecha) VALUES (3,3,3,202,'2022-09-22'::date);",
 "INSERT INTO faltar (id_empleado, id_curso, id_programa_curso, id_cliente, fecha) VALUES (3,3,3,202,'2022-09-29'::date);",
 "INSERT INTO faltar (id_empleado, id_curso, id_programa_curso, id_cliente, fecha) VALUES (4,4,4,203,'2022-11-16'::date);"]

In [22]:
fecha_reservacion_operaciones_inserts = get_sql_insert_statements_from_dataframe(fecha_reservacion_operaciones, "fecha_reservacion_operaciones")
display(fecha_reservacion_operaciones_inserts[:5])

["INSERT INTO fecha_reservacion_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, fecha) VALUES (1,2,1,1,'2022-09-01'::date);",
 "INSERT INTO fecha_reservacion_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, fecha) VALUES (1,2,1,1,'2022-09-02'::date);",
 "INSERT INTO fecha_reservacion_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, fecha) VALUES (1,2,1,1,'2022-09-03'::date);",
 "INSERT INTO fecha_reservacion_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, fecha) VALUES (1,2,1,1,'2022-09-05'::date);",
 "INSERT INTO fecha_reservacion_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, fecha) VALUES (1,2,1,1,'2022-09-06'::date);"]

In [23]:
laborar_operaciones_inserts = get_sql_insert_statements_from_dataframe(laborar_operaciones, "laborar_operaciones")
display(laborar_operaciones_inserts[:5])

['INSERT INTO laborar_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, id_empleado) VALUES (1,2,1,1,1);',
 'INSERT INTO laborar_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, id_empleado) VALUES (2,2,2,1,2);',
 'INSERT INTO laborar_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, id_empleado) VALUES (3,2,3,1,3);',
 'INSERT INTO laborar_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, id_empleado) VALUES (4,2,4,1,4);',
 'INSERT INTO laborar_operaciones (id_reservacion_operaciones, num_sala, num_piso, id_edificio, id_empleado) VALUES (5,2,5,1,5);']

In [24]:
DML_python_str = "\n\n".join([
    "\n".join(asistencia_inserts),
    "\n".join(registro_asistencia_inserts),
    "\n".join(horario_curso_inserts),
    "\n".join(asignar_inserts),
    "\n".join(faltar_inserts),
    "\n".join(fecha_reservacion_operaciones_inserts),
    "\n".join(laborar_operaciones_inserts),
])

In [25]:
file = open("../SQL/DML_python.sql", "w")
file.write(DML_python_str)
file.close()